# 2D monte carlo ray-tracing
This notebook contains a simple monte carlo ray-tracing solver for radiative transfer in a 2D atmosphere, including a small bit of theoretical background (see e.g. *Mayer, 2009: Radiative transfer in the cloudy atmosphere* for a more extensive overview)

First, we load the required python modules and read all atmospheric propertiesfrom the netcdf input file

In [33]:
import numpy as np
import netCDF4 as nc
import random as rd

nc_file = nc.Dataset("raytracing_input.nc")
k_ext   = nc_file.variables['k_ext'][:] # total extinction coefficient
k_sca     = nc_file.variables['k_sca'][:] # scattering coefficient
ccf     = nc_file.variables['ccf'][:]   # ratio of the cloud extinction coefficients over the total (cloud + clear sky extinction coefficient)

sza     = nc_file.variables['sza'][:]   # solar zenith angle
g       = nc_file.variables['g'][:]     # asymmetry parameters


## Scattering functions
When a photon hits a molecule and is scattered, we need to sample a random scattering angle from a probability distribution. We define two different scattering functions, depending on whether the photon is scattered by a cloud droplet or a gas molecule.

#### Henyey-Greenstein function
The Henyey-Greenstein phase function is an approximation of the very complex and expensive to calculate Mie scattering and often used for scattering by clouds. It is a probability density function for $\mu$, the cosine of scattering angle $\theta$ ($\mu=\cos\theta$)

$$P_{HG}\left(\mu\right)=\frac{1-g^2}{2\left(1+g^2-2g\mu\right)^{3/2}},$$

where $g$ is the asymmetry parameter. To be able to sample a random $\mu$, integrate $P_{HG}\left(\mu\right)$ from $-1$ to $\mu$ and solve for $\mu$:

$$\mu_{HG}=\dfrac{1}{2g}\left(1+g^2-\left(\dfrac{g^2-1}{2gr-g-1}\right)^2\right),$$

where $r$ is a random number between 0 and 1

In [34]:
def henyey(g=.85): #henyey greenstein scattering   
    r = rd.random()
    a = (1.-g**2)**2
    b = 2.*g*(2*r*g+1.-g)**2
    c = -g/2.-1./(2*g)
    return -1*(a/b)-c

#### Rayleigh function
For scattering by molecules, we use the Rayleigh phase function

$$P_{rayleigh} = \dfrac{3}{4}\left(1+\cos^2\theta\right),$$

and, after integrating from $-1$ to $\mu$ and solving for $\theta$,

$$\mu_{Rayleigh} = u-\frac{1}{u},\ u=\left(-q+\sqrt{1+q^2}\right)^{1/3}\ \mathrm{and}\ q=4r-2,$$

where r is a random number between 0 and 1.

In [35]:
def rayleigh(): 
    r = rd.random()
    q = -8*r+4
    d = 1+q*q/4
    u = (-q/2+np.sqrt(d))**(1/3.)
    return u-1/u

### Sampling optical depth

The decrease in radiative flux along a path in a non-transparent atmosphere is determined by the Lambert-Beer law, so the radiative flux after a distance $s$ from the top of the atmosphere is

$$I(s)=-I_0\exp{\tau}, \tag{1}$$ 

where $I_0$ is the radiative flux at the top of atmosphere and $\tau$ is the optical depth, which is the integral of the extinction coefficient ($k_{ext}$) over distance $s$: $\tau=\int k_{ext} ds$. (Note: the extinction coefficient is not a constant, but a function of wavelength, temperature, pressure, humidity, cloud thickness and more). 

As given by eq. 1, the fraction of photon surviving after a distance $\tau$ is $\frac{1}{e^\tau}$, which also means that a random photon has a $\frac{1}{e^\tau}$ chance of travelling further than a distance $\tau$ before hitting a molecule or cloud droplet. For each initialised photon, we sample a random $\tau$ from a probability distribution, such that after many photons the fraction of photons that will travel further than $\tau$ is $\frac{1}{e^\tau}$:

$$\tau = -\ln{\left(1-r\right)},$$ 

where $r$ is a random number between 0 and 1

In [10]:
def sample_tau():
    return -np.log(1-rd.random())